# Imports

In [1]:
import numpy as np
import luch_basics as lb
import luch_elements as le
import luch_graphics as lg

from bokeh.io import output_notebook,show, output_notebook, push_notebook, export_png
output_notebook()


Loading BokehJS ...

# Define beamline

In [4]:
# --------------------- particle definition---------------------------------
start1 = np.array([[0.000], [0.03], [0.000], [0.03], [0], [0]])  # create start position for 1-st particle
start2 = np.array([[0.003], [0.00], [0.003], [0.00], [0], [0]])  # create start position for 2-nd particle

proton1 = lb.Particle(1, 30, 1, start1)  # create first particle
proton2 = lb.Particle(1, 30, 1, start2)  # create second particle


# ------------------------- beamline definition---------------------------

# ------------------------- define drifts---------------------------

drift_length_1 = 0.9 #[m]  drift before magnet
                                  
drift_length_2 = 1.1 # [m]  drift before accelerator

drift_length_3 = 0.35 # [m]  drift before quad

drift_length_4 = 4.3 # [m] drift after last optical element

drift_before_magnet = le.driftspace(length=drift_length_1)
drift_before_accelerator = le.driftspace(length=drift_length_2)
drift_before_quad = le.driftspace(length=drift_length_3)
drift_before_end_chamber = le.driftspace(length=drift_length_4)


#  -------------------------create dipole magnet-----------------------------
dipole = le.dipole(radius=0.33, angle=90, gap=0.055, index=0, pfa1=30.615, pfa2=30.615, K=0.6)


#  -------------------------generate accelerator tube-------------------

# create variation of electrode opening
dr = -0.5 # [mm]
Radius = np.ones(40) * 0.030  # create a numpy array of equal 30 mm openings
Radius = np.array([Radius[i] + i * dr / 1000 for i in range(len(Radius))])

tube_lens_voltage = -56.0 # [kV] 
terminal_voltage = -460 # [kV]

Voltages_tube = np.linspace(0, terminal_voltage, 38)
Voltages_lens = np.array([0, tube_lens_voltage])
Voltages = np.concatenate((Voltages_lens, Voltages_tube), axis=None)

acc_electrode_gap = 0.025  # gap between electrodes in the accelerator tube

acc_tube = le.tube(r=Radius, V=Voltages, gap=acc_electrode_gap)  # generate accelerator tube


#  -------------------------generate quad triplet-------------------

quad_voltage = 5275 # [V]
quad_astigmatism = -9.6 # [%] 
quad1 = le.quad(r=0.025, V=quad_voltage * (1 + quad_astigmatism / 100), L=0.102, dir=1)
quad2 = le.quad(r=0.025, V=quad_voltage, L=0.175, dir=0)
quad3 = le.quad(r=0.025, V=quad_voltage * (1 + quad_astigmatism / 100), L=0.102, dir=1)

# -------------------------beamline assembly-------------------

beamline_list = [drift_before_magnet, dipole, drift_before_accelerator, acc_tube, drift_before_quad, quad1, quad2,
                 quad3, drift_before_end_chamber]

beamline = lb.BeamLine(beamline_list)

# Particle tracing

In [3]:
proton1 = lb.Particle(1, 30, 1, start1) #create first particle
proton2 = lb.Particle(1, 30, 1, start2) #create second particle

ray1 = beamline.trace(proton1)
ray2 = beamline.trace(proton2)

# ------------------------ graphics ---------------------

plot=lg.plot_trajectory([ray1,ray2])
show(plot)